In [5]:
!pip install numpy


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\tharu\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
RAW_DATA_FILE = 'raw_data/SME-divya-3p-1ms-influxdata_1apr2021-31dec2021.csv'
CLEANED_DATA_DIR = 'data'
CLEANED_DATA_FILE = os.path.join(CLEANED_DATA_DIR, 'cleaned_bangalore_data.csv')

In [3]:
print(f"Loading raw data from {RAW_DATA_FILE}...")
try:
    df = pd.read_csv(RAW_DATA_FILE, comment='#')
except FileNotFoundError:
    print(f"ERROR: Raw data file not found at {RAW_DATA_FILE}")
    print("Please make sure the file is in the 'raw_data' directory.")
    exit()

Loading raw data from raw_data/SME-divya-3p-1ms-influxdata_1apr2021-31dec2021.csv...


In [4]:
df.head()

,Unnamed: 0,result,table,_start,_stop,_time,_value,_field,_measurement
0,NaN,NaN,0,2020-12-31T18:30:00Z,2022-12-31T18:29:59Z,2021-04-06T16:00:00.246Z,0.87,pf,Phase3
1,NaN,NaN,0,2020-12-31T18:30:00Z,2022-12-31T18:29:59Z,2021-04-06T16:00:07.203Z,0.87,pf,Phase3
2,NaN,NaN,0,2020-12-31T18:30:00Z,2022-12-31T18:29:59Z,2021-04-06T16:00:14.167Z,0.88,pf,Phase3
3,NaN,NaN,0,2020-12-31T18:30:00Z,2022-12-31T18:29:59Z,2021-04-06T16:00:21.245Z,0.88,pf,Phase3
4,NaN,NaN,0,2020-12-31T18:30:00Z,2022-12-31T18:29:59Z,2021-04-06T16:00:29.223Z,0.87,pf,Phase3


In [5]:
if all(col in df.columns for col in ['_time', '_value', '_field', '_measurement']):
    df = df[['_time', '_value', '_field', '_measurement']]
    print("Successfully loaded and selected relevant columns.")
else:
    print("ERROR: The CSV file is missing one of the required columns: '_time', '_value', '_field', '_measurement'")
    exit()

print(f"Raw data shape: {df.shape}")
print(df.head())

Successfully loaded and selected relevant columns.
Raw data shape: (19954040, 4)
                      _time  _value _field _measurement
0  2021-04-06T16:00:00.246Z    0.87     pf       Phase3
1  2021-04-06T16:00:07.203Z    0.87     pf       Phase3
2  2021-04-06T16:00:14.167Z    0.88     pf       Phase3
3  2021-04-06T16:00:21.245Z    0.88     pf       Phase3
4  2021-04-06T16:00:29.223Z    0.87     pf       Phase3


In [6]:
print(f"Raw data shape: {df.shape}")
print(df.head())

Raw data shape: (19954040, 4)
                      _time  _value _field _measurement
0  2021-04-06T16:00:00.246Z    0.87     pf       Phase3
1  2021-04-06T16:00:07.203Z    0.87     pf       Phase3
2  2021-04-06T16:00:14.167Z    0.88     pf       Phase3
3  2021-04-06T16:00:21.245Z    0.88     pf       Phase3
4  2021-04-06T16:00:29.223Z    0.87     pf       Phase3


In [8]:
print("Pivoting data (long to wide format)... This may take a moment.")
try:
    df_pivoted = df.pivot_table(
        index='_time', 
        columns=['_measurement', '_field'], 
        values='_value'
    )
except Exception as e:
    print(f"ERROR during pivoting: {e}")
    print("There might be duplicate entries (same time, measurement, and field). Check data integrity.")
    exit()

print("Pivot complete.")
print(f"Pivoted data shape: {df_pivoted.shape}")
print(df_pivoted.head())

Pivoting data (long to wide format)... This may take a moment.
Pivot complete.
Pivoted data shape: (3330892, 18)
_measurement              Phase1                                       Phase2  \
_field                   current energy frequency    pf power voltage current   
_time                                                                           
2021-04-06T15:00:02.187Z     NaN    NaN       NaN   NaN   NaN     NaN    0.75   
2021-04-06T15:00:04.433Z     NaN    NaN       NaN   NaN   NaN     NaN     NaN   
2021-04-06T15:00:06.684Z     NaN   2.94      50.0  0.78  27.6   245.7     NaN   
2021-04-06T15:00:09.044Z     NaN    NaN       NaN   NaN   NaN     NaN    0.75   
2021-04-06T15:00:11.408Z     NaN    NaN       NaN   NaN   NaN     NaN     NaN   

_measurement                                                   Phase3         \
_field                   energy frequency    pf power voltage current energy   
_time                                                                         

In [9]:
print("Cleaning column names (e.g., ('Phase1', 'pf') -> 'Phase1_pf')...")

df_pivoted.columns = ['_'.join(col) for col in df_pivoted.columns.values]
print(f"New columns: {df_pivoted.columns.tolist()[:5]}...")

Cleaning column names (e.g., ('Phase1', 'pf') -> 'Phase1_pf')...
New columns: ['Phase1_current', 'Phase1_energy', 'Phase1_frequency', 'Phase1_pf', 'Phase1_power']...


In [10]:
print("Converting index to datetime (using ISO8601 format)...")

try:
    df_pivoted.index = pd.to_datetime(df_pivoted.index, format='ISO8601')
except Exception as e:
    print(f"ERROR converting datetime index: {e}")
    print("If this error persists, try: df_pivoted.index = pd.to_datetime(df_pivoted.index, format='mixed')")
    exit()

Converting index to datetime (using ISO8601 format)...


In [11]:
print("Resampling data to 1-hour intervals and imputing missing values...")
df_final = df_pivoted.resample('1H').mean()

df_final = df_final.ffill()

df_final = df_final.bfill() # Back-fill to handle NaNs at the start

print(f"Final cleaned data shape: {df_final.shape}")
print(df_final.head())

Resampling data to 1-hour intervals and imputing missing values...


C:\Users\dhanu\AppData\Local\Temp\ipykernel_23748\1910336210.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_final = df_pivoted.resample('1H').mean()


Final cleaned data shape: (3188, 18)
                           Phase1_current  Phase1_energy  Phase1_frequency  \
_time                                                                        
2021-04-06 15:00:00+00:00        1.113794       3.154319         49.990993   
2021-04-06 16:00:00+00:00        1.113794       3.268525         49.941452   
2021-04-06 17:00:00+00:00        2.562022       3.828176         49.827912   
2021-04-06 18:00:00+00:00        1.855851       4.298222         49.958247   
2021-04-06 19:00:00+00:00        1.704474       4.731263         50.000000   

                           Phase1_pf  Phase1_power  Phase1_voltage  \
_time                                                                
2021-04-06 15:00:00+00:00   0.829654    293.827483      245.147086   
2021-04-06 16:00:00+00:00   0.817635    256.130445      244.284450   
2021-04-06 17:00:00+00:00   0.836242    584.991429      238.038411   
2021-04-06 18:00:00+00:00   0.824433    422.856701      237.871164

In [12]:
print("Saving cleaned, model-ready data...")
os.makedirs(CLEANED_DATA_DIR, exist_ok=True)

df_final.to_csv(CLEANED_DATA_FILE)

print(f"--- Pipeline Complete ---")
print(f"Cleaned data saved to: {CLEANED_DATA_FILE}")

Saving cleaned, model-ready data...
--- Pipeline Complete ---
Cleaned data saved to: data\cleaned_bangalore_data.csv


In [13]:
df_final.head()

,Phase1_current,Phase1_energy,Phase1_frequency,Phase1_pf,Phase1_power,Phase1_voltage,Phase2_current,Phase2_energy,Phase2_frequency,Phase2_pf,Phase2_power,Phase2_voltage,Phase3_current,Phase3_energy,Phase3_frequency,Phase3_pf,Phase3_power,Phase3_voltage
_time,,,,,,,,,,,,,,,,,,
2021-04-06 15:00:00+00:00,1.113794,3.154319,49.990993,0.829654,293.827483,245.147086,0.279607,0.654827,49.991455,0.898014,134.743897,245.913626,1.645023,2.455394,49.989815,0.909368,361.782176,243.481019
2021-04-06 16:00:00+00:00,1.113794,3.268525,49.941452,0.817635,256.130445,244.284450,0.748005,0.736197,49.941080,0.736385,134.743897,244.346244,1.512787,2.828150,49.936300,0.909368,332.292037,242.144028
2021-04-06 17:00:00+00:00,2.562022,3.828176,49.827912,0.836242,584.991429,238.038411,0.272659,0.824220,49.828571,0.913978,49.951209,237.977802,0.947319,3.089429,49.819121,0.946044,211.166154,235.926374
2021-04-06 18:00:00+00:00,1.855851,4.298222,49.958247,0.824433,422.856701,237.871164,0.375438,0.906108,49.960052,0.871160,67.499485,237.161598,1.096744,3.315538,49.954103,0.953615,246.339487,235.372564
2021-04-06 19:00:00+00:00,1.704474,4.731263,50.000000,0.814737,389.253684,239.472527,0.478763,0.967316,50.000000,0.837184,86.323158,238.679474,0.980026,3.553087,49.999736,0.962480,224.555673,237.166491
